In [1]:
import pandas as pd
import json
from tqdm import tqdm
from collections import Counter

In [2]:
df1 = pd.read_csv('arxiv_output.csv')


In [3]:
df1.head()

,arxiv_id,named_entities
0,1,"[{""entity_group"": ""Method"", ""score"": 0.6655504..."
1,2,"[{""entity_group"": ""Method"", ""score"": 0.7490195..."
2,3,"[{""entity_group"": ""Method"", ""score"": 0.5377213..."
3,4,"[{""entity_group"": ""Task"", ""score"": 0.992919981..."
4,5,"[{""entity_group"": ""Method"", ""score"": 0.8687754..."


In [4]:
datalist = []
for index,row in tqdm(df1.iterrows(), total=df1.shape[0]):
    try:
        datalist.extend(json.loads(row['named_entities']))
    except Exception as e:
        print(e)
        print(index,row['named_entities'])
        break

100%|██████████| 255297/255297 [00:28<00:00, 9061.81it/s]


In [7]:
print(datalist[:3])

[{'entity_group': 'Method', 'score': 0.6655504703521729, 'word': 'NodeTrix', 'start': 0, 'end': 8}, {'entity_group': 'Method', 'score': 0.9968996644020081, 'word': 'Hybrid Representation', 'start': 10, 'end': 31}, {'entity_group': 'Task', 'score': 0.8594679236412048, 'word': 'Analyzing Social Networks', 'start': 36, 'end': 61}]


In [ ]:
entities = ['|_|'.join([item['word'], item['entity_group']]) for item in datalist]
counter = Counter(entities)
data = [key.split('|_|') + [counter[key]] for key in counter]

In [14]:
data[0:10]

[['NodeTrix', 'Method', 4],
 ['Hybrid Representation', 'Method', 5],
 ['Analyzing Social Networks', 'Task', 1],
 ['large', 'Task', 186],
 ['social networks', 'Material', 492],
 ['large', 'Material', 226],
 ['networks', 'Task', 725],
 ['systems', 'Generic', 3565],
 ['global structure of the network', 'OtherScientificTerm', 1],
 ['analysis of local communities', 'Task', 1]]

In [ ]:
dfoc = pd.DataFrame(data, columns=['word', 'method', 'oc_count'])
dfoc.head()

,word,method,oc_count
0,NodeTrix,Method,4
1,Hybrid Representation,Method,5
2,Analyzing Social Networks,Task,1
3,large,Task,186
4,social networks,Material,492


In [9]:
dfoc_sort = dfoc.sort_values(by='oc_count', axis=0, ascending=False)
dfoc_sort.head()

,word,method,oc_count
318,it,Generic,14744
99,method,Generic,13582
137,approach,Generic,12836
1542,methods,Generic,12764
501,model,Generic,12301


In [13]:
dfoc_sort_no_generic = dfoc_sort[dfoc_sort['method'] != 'Generic']
dfoc_sort_no_generic.head(50)

,word,method,oc_count
1037,accuracy,Metric,4351
590,of,Task,3687
10690,neural networks,Method,3650
2406,features,OtherScientificTerm,3454
12684,deep learning,Method,3394
473,graph,OtherScientificTerm,2974
7841,deep neural networks,Method,2703
2698,machine learning,Task,2602
3897,images,Material,2445
2975,robustness,Metric,2442


In [19]:
# Open stop words file and read content
with open("stopwords.txt", "r") as f:
    stop_words = f.readlines()

# Remove whitespace and newline characters from stop words
stop_words = [word.strip() for word in stop_words]

dfoc_sort_no_generic['isDelete'] = dfoc_sort_no_generic.apply(lambda item: item['word'] not in stop_words, axis=1)
dfoc_sort_no_generic_no_words = dfoc_sort_no_generic[dfoc_sort_no_generic['isDelete'] == True]
dfoc_sort_no_generic_no_words.head(100)

/environment/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,word,method,oc_count,isDelete
1037,accuracy,Metric,4351,True
10690,neural networks,Method,3650,True
2406,features,OtherScientificTerm,3454,True
12684,deep learning,Method,3394,True
473,graph,OtherScientificTerm,2974,True
...,...,...,...,...
32123,adversarial examples,Material,612,True
4340,artificial intelligence,Task,610,True
13469,unlabeled data,Material,602,True
27469,Convolutional Neural Networks,Method,600,True


In [17]:
lastData = dfoc_sort_no_generic_no_words.iloc[:, 0: 3].values.tolist()
lastData[0: 50]

[['accuracy', 'Metric', 4351],
 ['neural networks', 'Method', 3650],
 ['features', 'OtherScientificTerm', 3454],
 ['deep learning', 'Method', 3394],
 ['graph', 'OtherScientificTerm', 2974],
 ['deep neural networks', 'Method', 2703],
 ['machine learning', 'Task', 2602],
 ['images', 'Material', 2445],
 ['robustness', 'Metric', 2442],
 ['learning', 'Task', 2438],
 ['machine learning', 'Method', 2427],
 ['computer vision', 'Task', 2338],
 ['complexity', 'Metric', 2201],
 ['Deep Learning', 'Method', 1757],
 ['training', 'Task', 1685],
 ['neural network', 'Method', 1606],
 ['Learning', 'Task', 1507],
 ['classification', 'Task', 1474],
 ['data', 'Material', 1430],
 ['graphs', 'OtherScientificTerm', 1422],
 ['noise', 'OtherScientificTerm', 1364],
 ['machine learning models', 'Method', 1355],
 ['AI', 'Task', 1327],
 ['deep learning models', 'Method', 1323],
 ['inference', 'Task', 1213],
 ['performance', 'Metric', 1196],
 ['social media', 'Material', 1178],
 ['real - world applications', 'Task',